In [ ]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [ ]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [ ]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv'):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_100/dataset 100_1.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_2.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_3.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_4.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_5.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_6.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_7.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_8.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_9.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_10.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_11.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_12.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_13.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_14.csv
csv file is loa

In [ ]:
target_list = cts_list

# history size and future size
history_list = [10, 20, 30, 40]
future_list = [10, 20, 30]
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# supervised attention factor
delta = 1
att_type = 'linear'
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

iteration_list = [3,4]
for iteration in iteration_list:
    for history_size in history_list:
        for future_size in future_list:
            print(f"{iteration}th iteration")
            print(f"history size: {history_size}")
            print(f"future size: {future_size}")
            history_series = []
            future_series = []

            # data to series
            for i in range(len(target_list)):
                history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                            step, start_idx=0, end_idx=None)
                if not i:
                    history_series = history
                    future_series = future
                else:
                    history_series = np.concatenate([history_series, history], axis=0)
                    future_series = np.concatenate([future_series, future], axis=0)
            
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{model_num}"
            if not exists("./model", model_name, 'model'):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}")
            else:
                DATT_seq2seq_GRU.model = loadfile("./model", model_name, 'model')
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists('./result', model_name):
                savefile(test_result, './result', model_name)
            print("\n")
    print('\n\n')
print('end opitmization')

3th iteration
history size: 10
future size: 10
Epoch 1/10000
581/581 - 9s - loss: 0.2352 - val_loss: 0.1614 - 9s/epoch - 16ms/step
Epoch 2/10000
581/581 - 5s - loss: 0.1545 - val_loss: 0.1459 - 5s/epoch - 9ms/step
Epoch 3/10000
581/581 - 5s - loss: 0.1438 - val_loss: 0.1322 - 5s/epoch - 9ms/step
Epoch 4/10000
581/581 - 5s - loss: 0.1350 - val_loss: 0.1297 - 5s/epoch - 8ms/step
Epoch 5/10000
581/581 - 5s - loss: 0.1276 - val_loss: 0.1116 - 5s/epoch - 8ms/step
Epoch 6/10000
581/581 - 5s - loss: 0.1212 - val_loss: 0.1122 - 5s/epoch - 9ms/step
Epoch 7/10000
581/581 - 5s - loss: 0.1144 - val_loss: 0.1046 - 5s/epoch - 9ms/step
Epoch 8/10000
581/581 - 5s - loss: 0.1090 - val_loss: 0.1010 - 5s/epoch - 8ms/step
Epoch 9/10000
581/581 - 5s - loss: 0.1050 - val_loss: 0.0975 - 5s/epoch - 8ms/step
Epoch 10/10000
581/581 - 5s - loss: 0.1015 - val_loss: 0.0962 - 5s/epoch - 8ms/step
Epoch 11/10000
581/581 - 5s - loss: 0.0979 - val_loss: 0.0925 - 5s/epoch - 8ms/step
Epoch 12/10000
581/581 - 5s - loss: 0

INFO:tensorflow:Assets written to: ./model/10_10_1_50_1_50_datt_seq2seq_gru_3\assets


INFO:tensorflow:Assets written to: ./model/10_10_1_50_1_50_datt_seq2seq_gru_3\assets


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.932547,0.929228,0.962515,0.94143,4.216931,4.652152,3.018687,3.96259
1,0.854473,0.877838,0.903776,0.878696,6.154257,6.112773,4.835263,5.700764
2,0.752587,0.832637,0.825899,0.803708,7.915257,7.155482,6.50239,7.191043
3,0.648692,0.805187,0.767005,0.740295,9.306822,7.721072,7.520281,8.182725
4,0.554263,0.785425,0.739352,0.693013,10.380806,8.104377,7.951497,8.812227
5,0.471005,0.766227,0.736972,0.658068,11.199981,8.46057,7.985714,9.215422
6,0.401735,0.745662,0.743771,0.63039,11.797866,8.825626,7.880397,9.501296
7,0.342002,0.725256,0.746316,0.604525,12.343449,9.173175,7.840391,9.785672
8,0.292902,0.706322,0.739063,0.579429,12.810507,9.484457,7.951082,10.082015


csv file is saved to: ./result/10_10_1_50_1_50_datt_seq2seq_gru_3.csv


3th iteration
history size: 10
future size: 20
Epoch 1/10000
577/577 - 8s - loss: 0.3059 - val_loss: 0.2578 - 8s/epoch - 14ms/step
Epoch 2/10000
577/577 - 4s - loss: 0.2273 - val_loss: 0.2382 - 4s/epoch - 8ms/step
Epoch 3/10000
577/577 - 4s - loss: 0.2132 - val_loss: 0.2307 - 4s/epoch - 8ms/step
Epoch 4/10000
577/577 - 4s - loss: 0.2043 - val_loss: 0.2214 - 4s/epoch - 8ms/step
Epoch 5/10000
577/577 - 4s - loss: 0.1978 - val_loss: 0.2127 - 4s/epoch - 8ms/step
Epoch 6/10000
577/577 - 5s - loss: 0.1892 - val_loss: 0.2054 - 5s/epoch - 8ms/step
Epoch 7/10000
577/577 - 4s - loss: 0.1826 - val_loss: 0.1980 - 4s/epoch - 8ms/step
Epoch 8/10000
577/577 - 4s - loss: 0.1762 - val_loss: 0.1919 - 4s/epoch - 8ms/step
Epoch 9/10000
577/577 - 5s - loss: 0.1711 - val_loss: 0.1819 - 5s/epoch - 8ms/step
Epoch 10/10000
577/577 - 4s - loss: 0.1628 - val_loss: 0.1870 - 4s/epoch - 8ms/step
Epoch 11/10000
577/577 - 4s - loss: 0.1602 - val_

...

csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_4.csv


4th iteration
history size: 40
future size: 30
Epoch 1/10000
559/559 - 8s - loss: 0.3840 - val_loss: 0.3122 - 8s/epoch - 14ms/step
Epoch 2/10000
559/559 - 5s - loss: 0.2865 - val_loss: 0.2910 - 5s/epoch - 8ms/step
Epoch 3/10000
559/559 - 5s - loss: 0.2650 - val_loss: 0.2648 - 5s/epoch - 8ms/step
Epoch 4/10000
559/559 - 5s - loss: 0.2496 - val_loss: 0.2535 - 5s/epoch - 8ms/step
Epoch 5/10000
559/559 - 5s - loss: 0.2348 - val_loss: 0.2440 - 5s/epoch - 8ms/step
Epoch 6/10000
559/559 - 5s - loss: 0.2185 - val_loss: 0.2235 - 5s/epoch - 8ms/step
Epoch 7/10000
559/559 - 5s - loss: 0.2065 - val_loss: 0.2155 - 5s/epoch - 8ms/step
Epoch 8/10000
559/559 - 5s - loss: 0.1958 - val_loss: 0.1998 - 5s/epoch - 8ms/step
Epoch 9/10000
559/559 - 5s - loss: 0.1854 - val_loss: 0.1893 - 5s/epoch - 8ms/step
Epoch 10/10000
559/559 - 5s - loss: 0.1767 - val_loss: 0.1793 - 5s/epoch - 8ms/step
Epoch 11/10000
559/559 - 5s - loss: 0.1695 - val_

INFO:tensorflow:Assets written to: ./model/40_30_1_50_1_50_datt_seq2seq_gru_4\assets


INFO:tensorflow:Assets written to: ./model/40_30_1_50_1_50_datt_seq2seq_gru_4\assets


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.845978,0.880563,0.921975,0.882838,7.742055,6.076717,4.278665,6.032479
1,0.778696,0.831525,0.844711,0.81831,9.282256,7.217664,6.039153,7.513024
2,0.677618,0.792472,0.763131,0.744407,11.205632,8.011357,7.462145,8.893045
3,0.579906,0.760989,0.698112,0.679669,12.793722,8.598886,8.427554,9.940054
4,0.485913,0.730736,0.66776,0.628137,14.153305,9.128688,8.844089,10.708694
5,0.395781,0.703991,0.660862,0.586878,15.343137,9.573725,8.937661,11.284841
6,0.311935,0.681852,0.65833,0.550706,16.371673,9.927655,8.973072,11.757467
7,0.235698,0.663585,0.651496,0.516926,17.253484,10.210959,9.064664,12.176369
8,0.167745,0.64973,0.632201,0.483225,18.003049,10.421883,9.314325,12.579752


csv file is saved to: ./result/40_30_1_50_1_50_datt_seq2seq_gru_4.csv





end opitmization
